## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-14-06-28-56 +0000,nypost,Charlie Kirk’s alleged assassin Tyler Robinson...,https://nypost.com/2025/09/14/us-news/charlie-...
1,2025-09-14-06-27-37 +0000,nyt,Romania Reports Russian Drone in Its Airspace,https://www.nytimes.com/2025/09/14/world/europ...
2,2025-09-14-05-55-03 +0000,bbc,History-maker Crawford stuns Canelo in Vegas,https://www.bbc.com/sport/boxing/articles/c62n...
3,2025-09-14-05-31-16 +0000,startribune,Lynx begin WNBA playoffs as dominant league ch...,https://www.startribune.com/minnesota-lynx-wnb...
4,2025-09-14-05-22-13 +0000,nyt,"Hermeto Pascoal, Prolific Brazilian Composer, ...",https://www.nytimes.com/2025/09/14/arts/music/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,kirk,34
0,charlie,31
38,trump,19
161,new,10
492,russia,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
163,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...,101
75,2025-09-13-19-19-12 +0000,nypost,Nearly $3M raised for Charlie Kirk’s family fo...,https://nypost.com/2025/09/13/business/new-fun...,99
13,2025-09-14-03-44-53 +0000,nypost,Charlie Kirk’s alleged assassin Tyler Robinson...,https://nypost.com/2025/09/13/us-news/charlie-...,97
93,2025-09-13-17-24-02 +0000,nyt,Trump Escalates Attacks on Political Opponents...,https://www.nytimes.com/2025/09/13/us/politics...,97
61,2025-09-13-20-58-19 +0000,nypost,"Jasmine Crockett denies ‘wannabe Hitler,’ left...",https://nypost.com/2025/09/13/us-news/jasmine-...,97


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
163,101,2025-09-13-09-00-00 +0000,wsj,"Spencer Cox, Utah’s Republican governor, addre...",https://www.wsj.com/politics/policy/utah-spenc...
15,60,2025-09-14-03-00-00 +0000,wsj,Israeli Prime Minister Benjamin Netanyahu and ...,https://www.wsj.com/politics/national-security...
175,35,2025-09-13-07-10-14 +0000,bbc,New Nato mission to bolster eastern flank afte...,https://www.bbc.com/news/articles/c0q719qg1wko...
13,32,2025-09-14-03-44-53 +0000,nypost,Charlie Kirk’s alleged assassin Tyler Robinson...,https://nypost.com/2025/09/13/us-news/charlie-...
57,32,2025-09-13-21-32-26 +0000,nypost,Israel bombs Hamas operatives in another Gaza ...,https://nypost.com/2025/09/13/world-news/israe...
69,29,2025-09-13-20-08-47 +0000,nypost,Savage gunman chased 21-year-old into her Quee...,https://nypost.com/2025/09/13/us-news/heartbro...
18,28,2025-09-14-02-56-48 +0000,nypost,Warner Bros. Discovery shares spike as CEO Dav...,https://nypost.com/2025/09/13/business/warner-...
36,26,2025-09-13-23-55-41 +0000,latimes,California lawmakers pass measures to expand o...,https://www.latimes.com/california/story/2025-...
27,25,2025-09-14-01-00-00 +0000,wsj,The unemployment rate is low. But a measure th...,https://www.wsj.com/economy/jobs/underemployed...
75,24,2025-09-13-19-19-12 +0000,nypost,Nearly $3M raised for Charlie Kirk’s family fo...,https://nypost.com/2025/09/13/business/new-fun...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
